In [1]:
from importlib.metadata import version
from torch.utils.data import Dataset,DataLoader
import torch,tiktoken,torch.nn as nn
print(f"Torch version: {version('torch')}")

Torch version: 2.9.1


## Data Loader

In [2]:
class GPTDatasetV1(Dataset):
    def __init__(self,txt,tokenizer,max_length,stride):
        self.input_ids=[]
        self.target_ids=[]
        token_ids=tokenizer.encode(txt,allowed_special={'<|endoftext|>'})
        for i in range(0,len(token_ids)-max_length,stride):
            input_chunk=token_ids[i:i+max_length]
            target_chunk=token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self,idx):
        return self.input_ids[idx],self.target_ids[idx]
def create_dataloader(txt,batch_size=4,max_length=256,stride=128,shuffle=True):
    tokenizer=tiktoken.get_encoding('gpt2')
    dataset=GPTDatasetV1(txt,tokenizer,max_length,stride)
    dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=shuffle)
    return dataloader
with open('small-text-sample.txt','r',encoding='utf-8') as f:
    raw_text=f.read()
tokenizer=tiktoken.get_encoding('gpt2')
encoded_text=tokenizer.encode(raw_text)
vocab_size=50257
output_dim=256
max_len=1024
context_length=max_len
token_embedding_layer=nn.Embedding(vocab_size,output_dim)
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)
max_length=4
dataloader=create_dataloader(raw_text,batch_size=8,max_length=max_length,stride=max_length)
for batch in dataloader:
    x,y=batch
    token_embeddings=token_embedding_layer(x)
    pos_embeddings=pos_embedding_layer(torch.arange(max_length))
    input_embeddings=token_embeddings+pos_embeddings
    break
input_embeddings.shape

torch.Size([8, 4, 256])

# Multi-Head Attention
## Alternative Implementation

In [3]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super().__init__()
        assert d_out%num_heads==0,'d_out must be divisible by num_heads.'
        self.d_out=d_out
        self.num_heads=num_heads
        self.head_dim=d_out//num_heads
        self.W_query=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_key=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.W_value=nn.Linear(d_in,d_out,bias=qkv_bias)
        self.out_proj=nn.Linear(d_out,d_out)
        self.dropout=nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
    def forward(self,x):
        b,num_tokens,d_in=x.shape
        keys=self.W_key(x)
        queries=self.W_query(x)
        values=self.W_value(x)
        keys=keys.view(b,num_tokens,self.num_heads,self.head_dim) 
        values=values.view(b,num_tokens,self.num_heads,self.head_dim)
        queries=queries.view(b,num_tokens,self.num_heads,self.head_dim)
        keys=keys.transpose(1,2)
        queries=queries.transpose(1,2)
        values=values.transpose(1,2)
        attn_scores=queries@keys.transpose(2,3)
        mask_bool=self.mask.bool()[:num_tokens,
                                   :num_tokens]
        attn_scores.masked_fill_(mask_bool,-torch.inf)
        attn_weights=torch.softmax(attn_scores/keys.shape[-1]**.5,dim=-1)
        attn_weights=self.dropout(attn_weights)
        context_vec=(attn_weights@values).transpose(1,2) 
        context_vec=context_vec.contiguous().view(b,num_tokens,self.d_out)
        context_vec=self.out_proj(context_vec)
        return context_vec
context_length=max_length
d_in=output_dim
d_out=d_in
mha=MultiHeadAttention(d_in,d_out,context_length,0,num_heads=2)
batch=input_embeddings
context_vecs=mha(batch)
print(f'context_vecs.shape: {context_vecs.shape}')

context_vecs.shape: torch.Size([8, 4, 256])
